In [1]:
import numpy as np 
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw, CanonSmiles
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

import csv
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as skm
from collections import defaultdict

import copy
import os
import pickle
from rich import print as rprint

# Train and Save Random Forest Models

## Functions

In [45]:
def get_train_data(pdict, tau):
    df= pd.read_csv(pdict['path'])
    molecules = [Chem.MolFromSmiles(s) for s in df[pdict['col']] ]
    fingers = [Chem.RDKFingerprint(x) for x in molecules]
    X = np.stack([np.array(x) for x in fingers])
    y = np.array(df[tau])
    return X, y

def get_test_data(pdict):
    df= pd.read_csv(pdict['path'])
    molecules = [Chem.MolFromSmiles(s) for s in df[pdict['col']] ]
    fingers = [Chem.RDKFingerprint(x) for x in molecules]
    X = np.stack([np.array(x) for x in fingers])
    return X

def train_test_split(X,y,train_frac=0.9):
    n = len(y)
    indx = np.random.permutation(n)
    ntr = int(np.floor(n*train_frac))
    Xtrain, Xtest = X[indx[0:ntr],:], X[indx[ntr:],:]
    ytrain, ytest = y[indx[0:ntr]], y[indx[ntr:]]
    return Xtrain, ytrain, Xtest, ytest

def train_and_save(X,y, tau, split_prob = 0.9, **kwargs):
    Xtrain, ytrain, Xtest, ytest = train_test_split(X,y, split_prob)
    
    yb_train = ytrain.astype(int) # binarized 0/1 partition ratio
    yb_test = ytest.astype(int)

    cls = RandomForestClassifier(**kwargs)
    cls.fit(Xtrain,yb_train)

    ybh_test = cls.predict(Xtest)
    acc_test = np.mean((yb_test==ybh_test).astype(int))

    ph_test = cls.predict_proba(Xtest)[:,1] # prob y=1    
    score = skm.roc_auc_score(yb_test,ph_test)
    output = {'acc': acc_test, 'auc': score}
    print('-- Validation Results {}: {}'.format(tau, output))
    return cls

def train(train_dataset_dict):
    classifiers = {}
    for protein, pdict in train_dataset_dict.items():
        print('Training on {}'.format(protein))
        th_set = pdict['thresholds']
        abbrevs = pdict['abbreviations']
        for tau, abv in zip(th_set, abbrevs):
            X,y = get_train_data(pdict, tau)
            classifiers['{}_{}'.format(protein, abv)] = train_and_save(X,y, tau, split_prob, **myargs)
        print()
    return classifiers
    
def test(dataset_dict, classifiers):
    results = {protein: {} for protein in dataset_dict.keys()}
    for protein, pdict in dataset_dict.items():
        print('Testing on {}'.format(protein))
        X = get_test_data(pdict)
        for cls_name, cls in classifiers.items():
            results[protein][cls_name] = classifiers[cls_name].predict(X) 
            # classifiers[cls_name].predict_proba(X) to get probabilities
    return results

def ensemble_chemprop_rf(train_dataset_dict, test_dataset_dict, rf_results, use_random_forest_preds = True):
    ensemble_preds= {}
    for setname, protein_results in rf_results.items():
        for task_name, rf_preds in protein_results.items():
            
            protein, abv = task_name.split('_')
            
            dl_filepath = os.path.join(test_dataset_dict[setname]['dl_predictions_dir'], '{}.csv'.format(task_name.lower()) ) 
            if not os.path.exists(dl_filepath):
                rprint(f"{dl_filepath} NOT FOUND")
                continue 
            
            dl_pred = pd.read_csv(dl_filepath)
            i = train_dataset_dict[protein]['abbreviations'].index(abv)
            k = train_dataset_dict[protein]['thresholds'][i]
            tau = train_dataset_dict[protein]['numerical_thresholds'][i]
            dl_pred = np.array(dl_pred[k]) > 0.5
            if use_random_forest_preds:
                combo_pred = np.logical_and(rf_preds, dl_pred).tolist()
            else:
                combo_pred = dl_pred.tolist()
            ensemble_preds['{}>{}'.format(protein, tau)] = combo_pred
    return ensemble_preds

In [37]:
TRAIN_DATASET = {
    'MED1': {
        'path': 'YoungLab/droplets0521/MED1_dataset.csv',
        'thresholds': ['Threshold1', 'Threshold2', 'Threshold3'],
        'numerical_thresholds': [8.0, 5.4, 2.7],
        'abbreviations': ['tau1', 'tau2', 'tau3'],
        'col': 'SMILES'
        },
    'NPM1': {
        'path': 'YoungLab/droplets0521/NPM1_dataset.csv',
        'thresholds': ['Threshold1', 'Threshold2', 'Threshold3'],
        'numerical_thresholds': [4.5, 3.5, 2.7],
        'abbreviations': ['tau1', 'tau2', 'tau3'],
        'col': 'SMILES'
        },
    'HP1a': {
        'path': 'YoungLab/droplets0521/HP1a_dataset.csv',
        'thresholds': ['Threshold1', 'Threshold2', 'Threshold3'],
        'numerical_thresholds': [3.4, 2.7, 2.0],
        'abbreviations': ['tau1', 'tau2', 'tau3'],
        'col': 'SMILES'
        },
    'Nucleocapsid': {
        'path': 'YoungLab/droplets0521/Nucleocapsid_dataset.csv',
        'thresholds': ['HighThreshold', 'MediumThreshold', 'LowThreshold'],
        'numerical_thresholds': ['HighThreshold', 'MediumThreshold', 'LowThreshold'],
        'abbreviations': ['high', 'med', 'low'],
        'col': 'SMILES'
        }
}

## Train

In [13]:
myargs = {"n_estimators":200, "oob_score":True,\
          "min_samples_leaf":2, "n_jobs":4}
split_prob = 0.9

In [14]:
classifiers = train(TRAIN_DATASET)

Training on MED1
-- Validation Results Threshold1: {'acc': 0.991304347826087, 'auc': 0.7719298245614035}
-- Validation Results Threshold2: {'acc': 0.8173913043478261, 'auc': 0.8746666666666667}
-- Validation Results Threshold3: {'acc': 0.8521739130434782, 'auc': 0.8572841133816743}

Training on NPM1
-- Validation Results Threshold1: {'acc': 0.9716981132075472, 'auc': 0.45714285714285713}
-- Validation Results Threshold2: {'acc': 0.9245283018867925, 'auc': 0.9193262411347518}
-- Validation Results Threshold3: {'acc': 0.8773584905660378, 'auc': 0.8493647912885662}

Training on HP1a
-- Validation Results Threshold1: {'acc': 0.979381443298969, 'auc': 0.9791666666666666}
-- Validation Results Threshold2: {'acc': 0.8969072164948454, 'auc': 0.8144820295983088}
-- Validation Results Threshold3: {'acc': 0.7628865979381443, 'auc': 0.7978835978835979}

Training on Nucleocapsid
-- Validation Results HighThreshold: {'acc': 0.8333333333333334, 'auc': 0.8626453488372093}
-- Validation Results MediumT

In [17]:
pickle.dump(classifiers, open("YoungLab/droplets0521/droplet_models/random_forest.p", "wb"))

# Eval classifiers

## Load RandomForest Classifiers

In [7]:
classifiers = pickle.load(open("YoungLab/droplets0521/droplet_models/random_forest.p", "rb"))

In [ ]:
PreDATASET = {
    'MED1': {'path': 'YoungLab/droplets0521/MED1_dataset.csv',
             'key': 'SMILES'
            },
    'NPM1': {'path': 'YoungLab/droplets0521/NPM1_dataset.csv',
            'key': 'SMILES'
            },
    'HP1a': {'path': 'YoungLab/droplets0521/HP1a_dataset.csv',
            'key': 'SMILES'
            },
    'Nucleocapsid': {'path': 'YoungLab/droplets0521/Nucleocapsid_dataset.csv',
                    'key': 'SMILES'}
}

## Fluorophores

In [ ]:
TEST_DATASET = {
    'Fluorophores': {
        'path': 'YoungLab/droplets0521/fluorophores.csv',
        'col': 'SMILES',
        'dl_predictions_dir': 'YoungLab/Condensates_May2021/fluorophores_predictions/'}
}

In [ ]:
fluorophores = pd.read_csv(TEST_DATASET["Fluorophores"]["path"])

In [ ]:
# Check overlap with previous datasets
prev_smiles = []
for protein, pdict in PreDATASET.items():
    df = pd.read_csv(pdict['path'])
    smiles =  list(df[pdict['key']])
    prev_smiles.extend([CanonSmiles(s) for s in smiles])
prev_smiles = list(set(prev_smiles))

In [ ]:
fluorophore_set_smiles = [CanonSmiles(s) for s in list(fluorophores['SMILES'])]

prev_smiles_dict = {s: True for s in prev_smiles}
fluorophore_set_overlap = [prev_smiles_dict.get(s, False) for s in fluorophore_set_smiles]
print('Overlap between blue set and previous datasets: {}'.format(sum(fluorophore_set_overlap)))

In [ ]:
test_predictions = test(TEST_DATASET, classifiers)

In [ ]:
results = ensemble_chemprop_rf(TRAIN_DATASET, TEST_DATASET, test_predictions)

In [ ]:
pd.DataFrame(results).to_csv('YoungLab/Condensates_May2021/fluorophores_predict_062721.csv', index = False)

## Nucleocapsid

In [ ]:
PreDATASET.update({
    'Fluorophores': {'path': 'YoungLab/droplets0521/fluorophores.csv',
                    'key': 'SMILES'},
})

In [ ]:
TEST_DATASET = {
    'Nucleocapsid': {
        'path': 'YoungLab/Condensates_May2021/Nucleocapsid_dataset.csv',
        'col': 'SMILES',
        'thresholds': ['HighThreshold', 'MediumThreshold', 'LowThreshold'],
        'abbreviations': ['high', 'med', 'low'],
        'dl_predictions_dir': 'YoungLab/Condensates_May2021/fluorophores_predictions/nucleocapsid'}
}

In [ ]:
test_predictions = test(TEST_DATASET, classifiers)

In [ ]:
results = ensemble_chemprop_rf(TRAIN_DATASET, TEST_DATASET, test_predictions)

In [ ]:
pd.DataFrame(results).to_csv('YoungLab/Condensates_May2021/fluorophores_predict_080621.csv', index = False)

## FDA Drugs

In [ ]:
PreDATASET.update({
    'Nucleocapsid': {'path': 'YoungLab/droplets0521/Nucleocapsid_dataset.csv',
                    'key': 'SMILES'}
})

In [ ]:
TEST_DATASET = {
    'FDA': {
        'path': 'YoungLab/FDA/CorrectedFDA_Dataset.csv',
        'col': 'HARMONIZED_SMILES',
        'dl_predictions_dir': 'YoungLab/FDA/predictions/'}
}

In [ ]:
test_predictions = test(TEST_DATASET, classifiers)

In [ ]:
results = ensemble_chemprop_rf(TRAIN_DATASET, TEST_DATASET, test_predictions, use_random_forest_preds = False)

In [ ]:
results.to_csv('YoungLab/FDA/chemprop_predictions.csv', index = False)

## Blue Set

In [24]:
PreDATASET.update({
    'FDA': {'path': 'YoungLab/droplets0521/FDA/CorrectedFDA_Dataset.csv',
           'key': 'HARMONIZED_SMILES'},
})

In [11]:
TEST_DATASET = {
    'BlueSet': {
        'path': 'YoungLab/droplets0521/nuclei_nucleocapsid_blue_set_dataset.csv',
        'col': 'SMILES',
        'dl_predictions_dir': 'YoungLab/droplets0521/blue_set_predictions/'}
}

In [10]:
blue_set = pd.read_csv(TEST_DATASET["BlueSet"]["path"])

In [37]:
# Check overlap with previous datasets
prev_smiles = []
for protein, pdict in PreDATASET.items():
    df = pd.read_csv(pdict['path'])
    smiles =  list(df[pdict['key']])
    prev_smiles.extend([CanonSmiles(s) for s in smiles])
prev_smiles = list(set(prev_smiles))

In [42]:
blue_set_smiles = [CanonSmiles(s) for s in list(blue_set['SMILES'])]

In [46]:
prev_smiles_dict = {s: True for s in prev_smiles}
blue_set_overlap = [prev_smiles_dict.get(s, False) for s in blue_set_smiles]
print('Overlap between blue set and previous datasets: {}'.format(sum(blue_set_overlap)))

Overlap between blue set and previous datasets: 0


In [110]:
# TEST
test_predictions = test(TEST_DATASET, classifiers)

Testing on BlueSet


In [124]:
ens_preds = ensemble_chemprop_rf(TRAIN_DATASET, TEST_DATASET, test_predictions)

In [127]:
for k, v in ens_preds.items():
    print(k, sum(v))

MED1_>Threshold1 185
MED1_>Threshold2 197
MED1_>Threshold3 333
NPM1_>Threshold1 267
NPM1_>Threshold2 410
NPM1_>Threshold3 339
HP1a_>Threshold1 278
HP1a_>Threshold2 186
HP1a_>Threshold3 261
Nucleocapsid_>HighThreshold 16
Nucleocapsid_>MediumThreshold 23
Nucleocapsid_>LowThreshold 492


In [129]:
pd.DataFrame(ens_preds).to_csv('blue_set_predictions.csv')

## Drug Fluorophores

In [18]:
DRUG_FLUOROPHORES_DATASET = {
    'DrugFluorophores': {
        'path': 'YoungLab/droplets0521/more_fluorescent_drugs_072922.csv',
        'col': 'SMILES',
        'dl_predictions_dir': 'YoungLab/droplets0521/fluorescent_drugs_predictions/'}
}

In [20]:
drug_fluoro_test = pd.read_csv(DRUG_FLUOROPHORES_DATASET["DrugFluorophores"]["path"])

In [54]:
# Check overlap with previous datasets
prev_smiles = []
for protein, pdict in PreDATASET.items():
    df = pd.read_csv(pdict['path'])
    smiles =  list(df[pdict['key']])
    prev_smiles.extend([CanonSmiles(s) for s in smiles])
prev_smiles = list(set(prev_smiles))

In [55]:
drug_fluoro_smiles = [CanonSmiles(s) for s in list(drug_fluoro_test['SMILES'])]

In [56]:
prev_smiles_dict = {s: True for s in prev_smiles}
drug_fluoro_overlap = [prev_smiles_dict.get(s, False) for s in drug_fluoro_smiles]
print('Overlap between fluorescent drug set and previous datasets: {}'.format(sum(drug_fluoro_overlap)))

Overlap between fluorescent drug set and previous datasets: 7


In [57]:
[s for s in drug_fluoro_smiles if prev_smiles_dict.get(s, False) ]

['COC(=O)c1ccc2c(C(=Nc3ccc(N(C)C(=O)CN4CCN(C)CC4)cc3)c3ccccc3)c(O)[nH]c2c1',
 'Cc1[nH]c(/C=C2\\C(=O)Nc3ccccc32)c(C)c1CCC(=O)O',
 'Cc1ccc(C(=O)Nc2ccc(CN3CCN(C)CC3)c(C(F)(F)F)c2)cc1C#Cc1cnc2cccnn12',
 'COc1cc2nccc(Oc3ccc(NC(=O)C4(C(=O)Nc5ccc(F)cc5)CC4)cc3)c2cc1OC',
 'CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)/C=C/CN(C)C',
 'O=c1c(-c2ccc(O)cc2)coc2cc(O)ccc12',
 'O=c1c(-c2ccc(O)cc2)coc2cc(O)cc(O)c12']

In [65]:
# TEST
fluoro_drugs_predictions = test(DRUG_FLUOROPHORES_DATASET, classifiers)

Testing on DrugFluorophores


In [69]:
for k,v in fluoro_drugs_predictions['DrugFluorophores'].items():
    print(f"{k}, num positive {v.sum()}")

MED1_tau1, num positive 0
MED1_tau2, num positive 0
MED1_tau3, num positive 0
NPM1_tau1, num positive 0
NPM1_tau2, num positive 0
NPM1_tau3, num positive 0
HP1a_tau1, num positive 0
HP1a_tau2, num positive 0
HP1a_tau3, num positive 0
Nucleocapsid_high, num positive 0
Nucleocapsid_med, num positive 0
Nucleocapsid_low, num positive 44


## Lotto Plate 12/22

In [5]:
LOTTO_DATASET = {
    'Lotto': {
        'path': 'YoungLab/droplets0521/lotto_plate_structures_predictions_121622.csv',
        'col': 'SMILES',
        'dl_predictions_dir': 'YoungLab/droplets0521/lotto_121622_predictions/'}
}

In [6]:
lotto_test = pd.read_csv(LOTTO_DATASET["Lotto"]["path"])

In [12]:
# Check overlap with previous datasets
prev_smiles = []
for protein, pdict in PreDATASET.items():
    df = pd.read_csv(pdict['path'])
    smiles =  list(df[pdict['key']])
    prev_smiles.extend([CanonSmiles(s) for s in smiles])
prev_smiles = list(set(prev_smiles))

lotto_test_smiles = [CanonSmiles(s) for s in list(lotto_test['SMILES'])]

prev_smiles_dict = {s: True for s in prev_smiles}
lotto_fluoro_overlap = [prev_smiles_dict.get(s, False) for s in lotto_test_smiles]
print('Overlap between fluorescent drug set and previous datasets: {}'.format(sum(lotto_fluoro_overlap)))

Overlap between fluorescent drug set and previous datasets: 0


In [13]:
# TEST
lotto_test_predictions = test(LOTTO_DATASET, classifiers)

Testing on Lotto


In [14]:
lotto_test_predictions.keys()

dict_keys(['Lotto'])

In [16]:
for k,v in lotto_test_predictions['Lotto'].items():
    print(f"{k}, num positive {v.sum()}")

MED1_tau1, num positive 0
MED1_tau2, num positive 0
MED1_tau3, num positive 0
NPM1_tau1, num positive 0
NPM1_tau2, num positive 0
NPM1_tau3, num positive 0
HP1a_tau1, num positive 0
HP1a_tau2, num positive 1
HP1a_tau3, num positive 18
Nucleocapsid_high, num positive 0
Nucleocapsid_med, num positive 11
Nucleocapsid_low, num positive 202


In [19]:
lotto_test_predictions['Lotto'].keys()

dict_keys(['MED1_tau1', 'MED1_tau2', 'MED1_tau3', 'NPM1_tau1', 'NPM1_tau2', 'NPM1_tau3', 'HP1a_tau1', 'HP1a_tau2', 'HP1a_tau3', 'Nucleocapsid_high', 'Nucleocapsid_med', 'Nucleocapsid_low'])

In [ ]:
collected_results = ensemble_chemprop_rf(TRAIN_DATASET, LOTTO_DATASET, lotto_test_predictions, use_random_forest_preds = False)

In [49]:
pd.DataFrame(collected_results).to_csv("lotto.csv", index=False)